In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_poject').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression 

In [4]:
data = spark.read.csv('data/cruise_ship_info.csv',
                     header=True,
                     inferSchema=True)

In [5]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [7]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
for item in data.head(1)[0]:
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


# Converting Categorical

In [43]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [44]:
from pyspark.ml.feature import StringIndexer

In [45]:
indexer = StringIndexer(inputCol='Cruise_line',outputCol="cruise_cat")
indexed = indexer.fit(data).transform(data)
indexed.head(5)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0),
 Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0),
 Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)]

In [46]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [47]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [48]:
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density','cruise_cat'],
outputCol='features')

In [50]:
output = assembler.transform(indexed)

In [51]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruise_cat: double (nullable = false)
 |-- features: vector (nullable = true)



In [52]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [53]:
output.head()

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))

In [54]:
final_data = output.select('features','crew')
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [55]:
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [57]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               110|
|   mean| 8.155909090909098|
| stddev|3.6090112233142673|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [58]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               48|
|   mean|6.965208333333332|
| stddev|3.128401800922347|
|    min|             0.88|
|    max|             13.6|
+-------+-----------------+



In [59]:
lr = LinearRegression(labelCol='crew')

In [60]:
lr_model = lr.fit(train_data)

In [61]:
test_res = lr_model.evaluate(test_data)

In [62]:
test_res.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.5600454979267075|
| -0.8868468260446196|
|-0.13656323067597675|
|  0.5949753160370506|
|  1.7361575945806518|
| -0.2184758354189862|
|-0.19658313104277436|
|0.029248025595601135|
|  0.2449548926817009|
| -0.5522056991941326|
|  1.1092390118263893|
| 0.03839934461343475|
|  1.0535514280634342|
| 0.11270983267352719|
|  0.9491064004169552|
|-0.29095430008908707|
|  0.3930383152027326|
|  0.2372275101351251|
| -0.9336556519956378|
| 0.27194862832217126|
+--------------------+
only showing top 20 rows



In [63]:
test_res.rootMeanSquaredError

0.8948896419857361

In [64]:
test_res.r2

0.9164325235560262

In [65]:
final_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [66]:
unlabed_data = test_data.select('features')

In [67]:
unlabed_data.show()

+--------------------+
|            features|
+--------------------+
|[5.0,122.0,28.5,1...|
|[6.0,30.276999999...|
|[7.0,158.0,43.7,1...|
|[9.0,81.0,21.44,9...|
|[10.0,46.0,7.0,6....|
|[10.0,58.825,15.6...|
|[10.0,68.0,10.8,7...|
|[10.0,91.62700000...|
|[11.0,58.6,15.66,...|
|[11.0,85.0,18.48,...|
|[11.0,90.0,22.4,9...|
|[11.0,91.62700000...|
|[11.0,108.977,26....|
|[12.0,42.0,14.8,7...|
|[12.0,91.0,20.32,...|
|[13.0,85.619,21.1...|
|[14.0,30.27699999...|
|[14.0,33.0,4.9,5....|
|[14.0,63.0,14.4,7...|
|[14.0,83.0,17.5,9...|
+--------------------+
only showing top 20 rows



In [68]:
pred = lr_model.transform(unlabed_data)

In [69]:
pred.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,122.0,28.5,1...| 6.139954502073293|
|[6.0,30.276999999...| 4.436846826044619|
|[7.0,158.0,43.7,1...|13.736563230675976|
|[9.0,81.0,21.44,9...|  9.40502468396295|
|[10.0,46.0,7.0,6....| 2.733842405419348|
|[10.0,58.825,15.6...| 7.218475835418986|
|[10.0,68.0,10.8,7...| 6.556583131042775|
|[10.0,91.62700000...| 8.970751974404399|
|[11.0,58.6,15.66,...| 7.355045107318299|
|[11.0,85.0,18.48,...| 8.552205699194133|
|[11.0,90.0,22.4,9...|  9.89076098817361|
|[11.0,91.62700000...| 8.961600655386565|
|[11.0,108.977,26....|10.946448571936566|
|[12.0,42.0,14.8,7...| 6.687290167326473|
|[12.0,91.0,20.32,...| 9.040893599583045|
|[13.0,85.619,21.1...| 9.490954300089086|
|[14.0,30.27699999...|3.3369616847972674|
|[14.0,33.0,4.9,5....| 3.002772489864875|
|[14.0,63.0,14.4,7...| 6.543655651995638|
|[14.0,83.0,17.5,9...| 9.178051371677828|
+--------------------+------------

In [70]:
test_res.meanAbsoluteError 

0.6021756643542665

In [71]:
from pyspark.sql.functions import corr 

In [72]:
data.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

